In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise
from lib.environment.defensor.environment import Environment

import numpy as np
import os
from datetime import datetime

from lib.utils.configuration_utils import ConfigurationUtils

In [ ]:
task_training_name = "defensor"

In [ ]:
def create_env():
    def _init():
        return Environment()
    return _init

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
def get_task_models_path():
    current_datetime = datetime.now()

    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    minute = current_datetime.minute
    second = current_datetime.second

    datetime_name = f"{year}_{month}_{day}_{hour}_{minute}_{second}"

    return f"models/{task_training_name}/{datetime_name}"

In [ ]:
save_path = get_task_models_path()

create_folder_if_not_exists(save_path)

In [ ]:
num_threads = 10
model_filename_prefix = "DDPG_model"

In [ ]:
env = SubprocVecEnv([create_env() for i in range(num_threads)])

n_actions = env.action_space.shape[-1]
n_actions

In [ ]:
time_step = ConfigurationUtils.get_rsoccer_training_time_step_seconds()

In [ ]:
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=.1 * np.ones(n_actions), dt=time_step)

In [ ]:
tau = 0.005
gamma = 0.99
learning_rate = 0.001
policy = "MlpPolicy"
buffer_size = 5_000_000
batch_size = 128

In [ ]:
load_model = True
loaded_model_path = "models/defensor/2024_5_25_13_32_29/DDPG_model"

In [ ]:
model = DDPG(
    policy=policy,
    env=env,
    tau=tau,
    gamma=gamma,
    action_noise=action_noise,
    learning_rate=learning_rate,
    buffer_size=buffer_size,
    batch_size=batch_size)

if load_model:
    model.set_parameters(loaded_model_path)

In [ ]:
total_timesteps = 2000000

In [ ]:
saved_model_number = 10
save_freq = total_timesteps // (saved_model_number * num_threads)
log_interval = total_timesteps // 10

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=save_freq,
    save_path=save_path,
    name_prefix=model_filename_prefix)

In [ ]:
model.learn(
    total_timesteps=total_timesteps,
    log_interval=log_interval,
    callback=checkpoint_callback,
    progress_bar=True)

In [ ]:
model.save(f"{save_path}/DDPG_model")